# Convert ROS calibration files to anycam optics definitions

In [ ]:
import os

import yaml
import json

import numpy as np

## Input

- ROS calibration file in yaml format
- sensor pixel dimensions in um

In [ ]:
ros_calibration_filename = "input/calibration_50-0503433255.yaml"
# sensor
pixel_size_um_x = 3.45

## Output

In [ ]:
output_dir = "output"

optic_dir = "pingen"
sensor_dir = "sensor"
camera_dir = "camera"

anycam_optic_filename = "lens.json"
anycam_sensor_filename = "sensor.json"
anycam_camera_filename = "camera.json"

In [ ]:
with open(ros_calibration_filename, "r") as file:
    ros_calibration = yaml.safe_load(file)

In [ ]:
# normalization by sensor width (in mm)
sensor_width = pixel_size_um_x * ros_calibration["image_width"] * 1e-3
normalization_factor = sensor_width / (pixel_size_um_x * 1e-3)

In [ ]:
normalization_factor

## Optics

In [ ]:
lens_output = {
    "sDTI": "/anycam/db/project/pingen/opencv:1.0",
    "sDTI/doc": "The calibration parameters used are those of the opencv pinhole camera calibration model: c.f. https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d",
    "sId": "${filebasename}",
    # "sInputType": "radius/normalized/fixed/mm",
    # "sOutputType": "angle/rad",
    # "fMaxAngle_deg": 180.0,
    "sUnit": "pixel",
    "sUnit/doc": "The measurement unit of the calibration data. Has to be one of ['pixel', 'mm']",
}
normalization_factor = 1.0

lens_output[
    "sSensorName"
] = f"{ros_calibration['camera_name']} {ros_calibration['image_width']}x{ros_calibration['image_height']} - {pixel_size_um_x}um"
lens_output["sLensName"] = f"{ros_calibration['camera_name']}"

image_width = ros_calibration["image_width"]
image_height = ros_calibration["image_height"]
lens_output["lSenSizeXY"] = [image_width, image_height]
lens_output["lSenSizeXY/doc"] = "sensor size in units determined by 'sUnit'"

fx = ros_calibration["camera_matrix"]["data"][0] * normalization_factor
fy = ros_calibration["camera_matrix"]["data"][4] * normalization_factor
lens_output["lFocLenXY"] = [fx, fy]

cx = ros_calibration["camera_matrix"]["data"][2] * normalization_factor
cy = ros_calibration["camera_matrix"]["data"][5] * normalization_factor
lens_output["lImgCtrXY"] = [cx, cy]

# vignetting is not defined in ROS calibration file
lens_output["fVignettingNorm"] = 1.0
lens_output["fVignettingNorm/doc"] = (
    "image plane positions in the units given by 'sUnit' are divided by this value before applied to the vignetting polynomial",
)

lens_output["lVignetting"] = [0.0, 0.0, 0.0]
lens_output["lVignetting/doc"] = "vignetting polynomial"


if ros_calibration["distortion_model"] == "plumb_bob":
    lens_output["distortion_model"] = "plumb_bob"

    lens_output[
        "lDistRad/doc"
    ] = "2, 3 or 6 radial distortion coefficients. Undefined coefficients are assumed to be zero."
    lens_output["lDistRad"] = []
    # k1
    lens_output["lDistRad"].append(
        ros_calibration["distortion_coefficients"]["data"][0]
        / np.power(normalization_factor, 3)
    )
    # k2
    lens_output["lDistRad"].append(
        ros_calibration["distortion_coefficients"]["data"][1]
        / np.power(normalization_factor, 5)
    )
    # k3
    lens_output["lDistRad"].append(
        ros_calibration["distortion_coefficients"]["data"][4]
        / np.power(normalization_factor, 7)
    )

    lens_output["lDistTan/doc"] = "2 tangential distortion coefficients"
    lens_output["lDistTan"] = []
    # p1
    lens_output["lDistTan"].append(
        ros_calibration["distortion_coefficients"]["data"][2]
        / np.power(normalization_factor, 2)
    )
    # p2
    lens_output["lDistTan"].append(
        ros_calibration["distortion_coefficients"]["data"][3]
        / np.power(normalization_factor, 4)
    )

    # for completness set other parameters and docstrings
    lens_output["lDistTan/doc"] = "2 tangential distortion coefficients"
    lens_output["lDistPrism"] = []
    lens_output[
        "lDistPrism/doc"
    ] = "1 to 4 prism distortion coefficients. c.f. https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d"
    lens_output["lDistTilt"] = []
    lens_output["lDistTilt/doc"] = "2 sensor tilt angles in radians"

elif ros_calibration["distortion_model"] == "rational_polynomial":
    print(f'Distortion model {ros_calibration["distortion_model"]} not implemented')
elif ros_calibration["distortion_model"] == "equidistant":
    print(f'Distortion model {ros_calibration["distortion_model"]} not implemented')
else:
    print(f'Unknown distortion model {ros_calibration["distortion_model"]}')

In [ ]:
if not os.path.isdir(output_dir + "/" + optic_dir):
    os.makedirs(output_dir + "/" + optic_dir)

with open(os.path.join(output_dir, optic_dir, anycam_optic_filename), "w") as outfile:
    json.dump(lens_output, outfile, indent=4)

## Sensor

In [ ]:
sensor_output = {
    "sDTI": "/anycam/db/sensor/2d/std:1.0",
    "sId": "${filebasename}",
}
sensor_output["iPixCntX"] = ros_calibration["image_width"]
sensor_output["iPixCntY"] = ros_calibration["image_height"]
sensor_output["fPixSize"] = pixel_size_um_x
sensor_output[
    "sName"
] = f"{ros_calibration['camera_name']} {sensor_output['iPixCntX']}x{sensor_output['iPixCntY']} - {sensor_output['fPixSize']}um"

In [ ]:
if not os.path.isdir(output_dir + "/" + sensor_dir):
    os.makedirs(output_dir + "/" + sensor_dir)


with open(os.path.join(output_dir, sensor_dir, anycam_sensor_filename), "w") as outfile:
    json.dump(sensor_output, outfile, indent=4)

## Camera

In [ ]:
camera_output = {
    "sDTI": "/anycam/db/camera/pingen:1.1",
    "sId": "${filebasename}",
    "sInfo": "the projection refers to pinhole calibration data. The sensor pixel count given there has to agree with the pixel count of the sensor.",
    "fSensorScale": 1.0,
    "fSensorScale/doc": "Sensor scale in Blender scene. Can improve numerical accuracy, but will change resultant images for close objects.",
}

camera_output[
    "sName"
] = f"{ros_calibration['camera_name']} {sensor_output['iPixCntX']}x{sensor_output['iPixCntY']} - {sensor_output['fPixSize']}um"

if anycam_sensor_filename.endswith(".json"):
    anycam_sensor_filename = anycam_sensor_filename[:-5]
camera_output["sSensorId"] = (
    output_dir + "/" + sensor_dir + "/" + anycam_sensor_filename
)

if anycam_optic_filename.endswith(".json"):
    anycam_optic_filename = anycam_optic_filename[:-5]
camera_output["sProjectId"] = output_dir + "/" + optic_dir + "/" + anycam_optic_filename

In [ ]:
if not os.path.isdir(output_dir + "/" + camera_dir):
    os.makedirs(output_dir + "/" + camera_dir)

with open(os.path.join(output_dir, camera_dir, anycam_camera_filename), "w") as outfile:
    json.dump(camera_output, outfile, indent=4)